# Results Notebook

In [279]:
%load_ext autoreload
%autoreload 2

import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
df = pd.DataFrame
sns.set_theme()

def melt_row_col_data(arr,name):
    arr_df = df(arr)
    arr_df = arr_df.melt(
        ignore_index=False,
        var_name='Column',
        value_name='ADC_OUT_AMS',
    )
    arr_df['Row'] = arr_df.index
    return arr_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [280]:
# Generate the reference result

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)
wx = w @ x.T
wx_rounded = np.round(wx.T / 2**2).astype(int).T[::-1].T

[STIM_GEN] Generating random weights


### Comparing AMS vs RTL simulations of QRAcc

In [281]:
ams = np.loadtxt('../tb/qracc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')

ams_df = df(ams)
rtl_df = df(rtl)

# ams_df['Row'] = ams_df.index
# rtl_df['Row'] = rtl_df.index

ams_df = ams_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_AMS',
)
rtl_df = rtl_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_RTL',
)
ams_df['Row'] = ams_df.index
rtl_df['Row'] = rtl_df.index
# Join the two dataframes
adc_df = pd.merge(ams_df, rtl_df)

rtl_fp = rtl.flatten()/2**3
ams_fp = ams.flatten()/2**3

err =   rtl_fp - ams_fp
snr = 20*np.log(np.var(rtl_fp) / np.var(err))

rmse = np.sqrt(np.mean((rtl.flatten() - ams.flatten())**2))
fig = px.scatter(adc_df, y='ADC_OUT_AMS', x='ADC_OUT_RTL', width=400, height=400, title=f'AMS vs Shifted, RMSE={rmse:.2f}, SNR={snr:.2f}', labels={'ADC_OUT_AMS':'AMS','ADC_OUT_RTL':'RTL'})
fig.show()
fig.write_image('../images/rtl_vs_ams.svg')

/tmp/ipykernel_256777/2330861704.py:29: RuntimeWarning:

divide by zero encountered in scalar divide



# Sequential Accelerator Simulations

In [282]:
w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 4,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)

# ams = np.loadtxt('../tb/seq_acc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtlsims.txt')

wx = w @ x.T

# rtl.astype(int), wx[::-1].T

rtl_fp = rtl.flatten()/(2**8)
wx_fp = wx[::-1].T.flatten()/(2**8)

rmse = np.sqrt(np.mean((rtl_fp - wx_fp)**2))
# SNR

err =   rtl_fp - wx_fp
snr = 20*np.log(np.var(rtl_fp) / np.var(err))

px.scatter(x=rtl_fp, y=wx_fp, width=400, height=400, title=f'AMS vs Shifted, RMSE={rmse:.2f}, SNR={snr:.2f}', labels={'x':'RTL','y':'AMS'}).show()

[STIM_GEN] Generating random weights


In [283]:
q.simulate_trit_mul(w, x[0], trits = 4, verbose=True)

x trits:
     0  1  2  3
0   -1  0 -1 -1
1    1  1  0  0
2    0  0  0  0
3    0  1  1  0
4    1  0  1  0
..  .. .. .. ..
123 -1 -1 -1 -1
124  1  1  1  0
125  0  0 -1  0
126  0  0 -1 -1
127 -1  0  0 -1

[128 rows x 4 columns]
partials:
   0   1   2   3   4   5   6   7   8   9   ...  22  23  24  25  26  27  28  \
0 -16   8 -10  -2  -2  -4  -8  -4 -10   2  ...   4   6  22 -18  -2  12  20   
1 -13  15  -7  -1   5  -9  -1  -7  -3   7  ...  -1  -5   5  -1   5  13   7   
2  -5  -3 -13   1   9  -1   3 -13 -11   3  ...   3   3   7  -5  -3   7  13   
3 -13   9  -1  13   5   1  -1 -19   1   5  ...   5  -1  13  -9  -7  13   3   

   29  30  31  
0   4  -2  -6  
1   5  -3   1  
2  -3   1  -1  
3   1  -5   7  

[4 rows x 32 columns]
shifting accumulator:
    0    1    2    3   4   5   6    7    8   9   ...  22  23   24   25  26  \
0 -104   72   -8  104  40   8  -8 -152    8  40  ...  40  -8  104  -72 -56   
1  -92   12 -108   60  92  -4  20 -180  -84  44  ...  44  20  108  -76 -52   
2 -150  126 -11

array([ -39.,  -31.,   47.,  217.,  175.,   -9., -167.,  223.,   33.,
         39.,  -29.,   17., -139.,  -95.,   13.,  -95.,  -83., -259.,
         73., -157.,  -33.,  213.,   55., -113., -105.,  -63.,  -69.,
         27.,   -5., -135.,  127., -203.])

In [284]:
x[1] @ w.T

array([  25,  159,   65,   51,  103,  179,  -29,  211,  -53,  -43,  -25,
       -139,   89,  135, -137,    5,  -75,  125, -119,  -11,   -7,   39,
        -13,  -79,    5,   21,  143,  -17,    9, -107, -147,  -51])

In [285]:
df(rtl.astype(int)) 

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-114,85,-109,-11,23,-65,-63,-84,-91,55,...,39,33,97,-86,-9,105,101,47,-31,-39
1,-51,-95,-103,17,-17,105,21,3,-77,-1,...,-43,-41,97,-29,69,79,13,63,97,25
2,3,-31,-120,-27,-15,-19,41,-37,51,58,...,-79,-47,-9,-36,3,-15,-111,25,-31,31
3,-110,-57,-61,-47,89,-120,14,-120,-120,-30,...,97,-59,-116,25,49,105,17,95,17,97
4,-14,23,-102,-50,-107,-114,13,-3,-31,3,...,87,-25,69,-106,101,39,5,105,-103,-107
5,9,-84,19,47,15,57,-95,-19,-45,81,...,-49,55,47,59,-29,59,-23,3,95,-48
6,-20,26,-56,20,95,-80,16,-28,-110,-104,...,63,-16,87,80,-12,28,-20,56,16,18
7,-108,3,-10,56,28,-84,-24,-30,-76,30,...,73,-18,-44,6,-54,6,57,-10,57,-120
8,-25,105,-31,-57,53,-5,-69,-19,69,97,...,-80,-79,65,43,-19,-3,105,-106,-107,39
9,57,-87,21,-70,-85,77,61,-34,-77,103,...,-49,-120,-61,33,67,73,73,105,-94,-33


In [286]:
df(wx[::-1].T)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-203,127,-135,-5,27,-69,-63,-105,-113,55,...,39,33,223,-167,-9,175,217,47,-31,-39
1,-51,-147,-107,9,-17,143,21,5,-79,-13,...,-43,-53,211,-29,179,103,51,65,159,25
2,3,-11,-203,-25,-15,-19,45,-37,67,63,...,-119,-47,-9,-43,9,-15,-111,25,-39,31
3,-123,-89,-61,-91,129,-247,13,-165,-295,-63,...,145,-65,-179,25,51,171,21,111,11,227
4,-15,23,-115,-53,-121,-133,19,-1,-31,3,...,111,-25,69,-133,221,31,5,135,-135,-133
5,5,-155,19,91,15,57,-111,-19,-41,99,...,-57,51,49,59,-29,95,-23,3,105,-73
6,-28,26,-56,18,108,-88,16,-46,-142,-120,...,64,-16,146,84,2,32,-28,56,12,18
7,-138,10,-10,88,54,-92,-24,-38,-86,30,...,86,-18,-72,16,-58,2,60,-4,108,-148
8,-25,195,-35,-59,117,-5,-69,-19,93,185,...,-133,-87,85,43,-35,-3,137,-147,-129,87
9,71,-111,27,-111,-85,149,61,-43,-81,151,...,-49,-253,-85,33,99,75,119,183,-99,-37
